In [1]:
import numpy as np
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
data = np.genfromtxt('InverseGamma.csv', delimiter=',', skip_header=1)

In [3]:
n, Size = data.shape
train = data[:int(n*0.8), :]
test = data[int(n*0.8):, :]
test = test[test[:, 0].argsort()]

In [4]:
train_X = train[:, 2:]
train_Y = train[:, 0:2]
test_X = test[:, 2:]
test_Y = test[:, 0:2]

### Метод моментов

In [5]:
MM = np.zeros((data[:,0:2].shape))
for i in range(data.shape[0]):
    # Get mean and variance from data
    sample_mean = np.mean(data[i, 2:])
    sample_var = np.var(data[i, 2:])
    
    # For Inverse Gamma distribution:
    # E[X] = β/(α-1) for α > 1
    # Var[X] = β²/((α-1)²(α-2)) for α > 2
    
    # Method of moments estimation
    # From the mean and variance relationships:
    # m = β/(α-1)
    # v = β²/((α-1)²(α-2))
    # where m is sample_mean and v is sample_var
    
    # Solving for α using the ratio v/m²:
    # v/m² = 1/(α-2)
    ratio = sample_var/(sample_mean**2)
    alpha_estimate = 2 + 1/ratio
    
    # Then solve for β using the mean relationship:
    beta_estimate = sample_mean * (alpha_estimate - 1)
    
    # Ensure parameters are valid
    alpha_estimate = max(alpha_estimate, 2.1)  # alpha must be > 2 for variance to exist
    beta_estimate = max(beta_estimate, 0.0001)  # beta must be positive
    
    MM[i][0] = alpha_estimate
    MM[i][1] = beta_estimate

In [6]:
#Оценка качества регрессии первого параметра
mape = np.mean(np.abs((data[:,0] - MM[:,0]) / data[:,0])) * 100
r2 = r2_score(data[:,0], MM[:,0])
mse = mean_squared_error(data[:,0], MM[:,0])
rmse = np.sqrt(mean_squared_error(data[:,0], MM[:,0]))
print("\nMM par 1")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')

#Оценка качества регрессии второго параметра
mape = np.mean(np.abs((data[:,1] - MM[:,1]) / data[:,1])) * 100
r2 = r2_score(data[:,1], MM[:,1])
mse = mean_squared_error(data[:,1], MM[:,1])
rmse = np.sqrt(mean_squared_error(data[:,1], MM[:,1]))
print("\nMM par 2")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')


MM par 1
MSE: 0.09150261642275107
RMSE: 0.3024939940275692
R² Score: 0.5091820695816266
MAPE: 13.083975729354409%

MM par 2
MSE: 171.4852701303687
RMSE: 13.095238452596757
R² Score: -2059.1679787425783
MAPE: 13223.977787560267%


### Регрессия на интервалах

In [7]:
InverseGamma_inter = CatBoostRegressor(loss_function='MultiRMSE')
InverseGamma_inter.fit(train_X, train_Y, eval_set = (test_X, test_Y))

0:	learn: 0.5149848	test: 0.5156947	best: 0.5156947 (0)	total: 2.39s	remaining: 39m 45s
1:	learn: 0.5107446	test: 0.5116854	best: 0.5116854 (1)	total: 5.23s	remaining: 43m 30s
2:	learn: 0.5067383	test: 0.5076502	best: 0.5076502 (2)	total: 8.2s	remaining: 45m 25s
3:	learn: 0.5028696	test: 0.5037977	best: 0.5037977 (3)	total: 11.2s	remaining: 46m 34s
4:	learn: 0.4993136	test: 0.5002483	best: 0.5002483 (4)	total: 13.9s	remaining: 46m 12s
5:	learn: 0.4956998	test: 0.4966772	best: 0.4966772 (5)	total: 17.1s	remaining: 47m 20s
6:	learn: 0.4921619	test: 0.4932194	best: 0.4932194 (6)	total: 20.1s	remaining: 47m 37s
7:	learn: 0.4889223	test: 0.4900680	best: 0.4900680 (7)	total: 23s	remaining: 47m 30s
8:	learn: 0.4856148	test: 0.4868423	best: 0.4868423 (8)	total: 25.8s	remaining: 47m 17s
9:	learn: 0.4825087	test: 0.4837822	best: 0.4837822 (9)	total: 28.5s	remaining: 46m 57s
10:	learn: 0.4795291	test: 0.4808045	best: 0.4808045 (10)	total: 31.2s	remaining: 46m 43s
11:	learn: 0.4766385	test: 0.4779

In [8]:
Cat_predictions = InverseGamma_inter.predict(test_X)

In [9]:
InverseGamma_inter.save_model('InverseGamma_inter_model.cbm')

In [10]:
#Оценка качества регрессии первого параметра
mape = np.mean(np.abs((test_Y[:,0] - Cat_predictions[:,0]) / test_Y[:,0])) * 100
r2 = r2_score(test_Y[:,0], Cat_predictions[:,0])
mse = mean_squared_error(test_Y[:,0], Cat_predictions[:,0])
rmse = np.sqrt(mean_squared_error(test_Y[:,0], Cat_predictions[:,0]))
print("\nInter par 1")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')

#Оценка качества регрессии второго параметра
mape = np.mean(np.abs((test_Y[:,1] - Cat_predictions[:,1]) / test_Y[:,1])) * 100
r2 = r2_score(test_Y[:,1], Cat_predictions[:,1])
mse = mean_squared_error(test_Y[:,1], Cat_predictions[:,1])
rmse = np.sqrt(mean_squared_error(test_Y[:,1], Cat_predictions[:,1]))
print("\nInter par 2")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')


Inter par 1
MSE: 0.07726365280046656
RMSE: 0.27796340190835656
R² Score: 0.5820343531410269
MAPE: 10.906986227387796%

Inter par 2
MSE: 0.005904985842580568
RMSE: 0.07684390569577114
R² Score: 0.9306286206721349
MAPE: 11.73212620895081%


### Регрессия на статистиках

In [11]:
stat_M = np.zeros((7999,10)) # Матрицы для записиси статистик наблюдений
test_stat_M = np.zeros((2000,10))

In [12]:
#Далее для каждого наблюдения вычисляются 4 статистики
stat_M[:,0] = np.mean(train_X, axis=1) # Вычисляется среднее 
stat_M[:,1] = np.var(train_X, axis=1) # Вычисляется дисперсию 
stat_M[:,2] = np.std(train_X, axis=1) # Вычисляется стандартное отклонение
stat_M[:,3] =  stat_M[:,2] / stat_M[:,0] # Вычисляется вариацию 

quantiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95]
for i, q in enumerate(quantiles):
    stat_M[:, 4 + i] = np.quantile(train_X, q, axis=1)

In [13]:
test_stat_M[:,0] = np.mean(test_X, axis=1)
test_stat_M[:,1] = np.var(test_X, axis=1)
test_stat_M[:,2] = np.std(test_X, axis=1)
test_stat_M[:,3] =  test_stat_M[:,2] / test_stat_M[:,0]

quantiles = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95]
for i, q in enumerate(quantiles):
    test_stat_M[:, 4 + i] = np.quantile(test_X, q, axis=1)

In [14]:
InverseGamma_stat = CatBoostRegressor(iterations=10000,
                          learning_rate=0.05,
                          depth=5, loss_function='MultiRMSE')
InverseGamma_stat.fit(stat_M, train_Y, eval_set = (test_stat_M, test_Y))

0:	learn: 0.4985925	test: 0.4992246	best: 0.4992246 (0)	total: 1.68ms	remaining: 16.8s
1:	learn: 0.4793795	test: 0.4798271	best: 0.4798271 (1)	total: 2.97ms	remaining: 14.8s
2:	learn: 0.4613521	test: 0.4615150	best: 0.4615150 (2)	total: 4.13ms	remaining: 13.8s
3:	learn: 0.4445569	test: 0.4448335	best: 0.4448335 (3)	total: 5.65ms	remaining: 14.1s
4:	learn: 0.4283763	test: 0.4282596	best: 0.4282596 (4)	total: 7.11ms	remaining: 14.2s
5:	learn: 0.4121849	test: 0.4120603	best: 0.4120603 (5)	total: 8.36ms	remaining: 13.9s
6:	learn: 0.3972935	test: 0.3972106	best: 0.3972106 (6)	total: 9.78ms	remaining: 14s
7:	learn: 0.3828036	test: 0.3826698	best: 0.3826698 (7)	total: 11.1ms	remaining: 13.9s
8:	learn: 0.3697528	test: 0.3695352	best: 0.3695352 (8)	total: 12.7ms	remaining: 14.1s
9:	learn: 0.3573662	test: 0.3571654	best: 0.3571654 (9)	total: 14ms	remaining: 14s
10:	learn: 0.3451527	test: 0.3449152	best: 0.3449152 (10)	total: 15.2ms	remaining: 13.8s
11:	learn: 0.3339795	test: 0.3338218	best: 0.33

In [15]:
InverseGamma_stat.save_model('InverseGamma_stat_model.cbm')

In [16]:
Cat_predictions = InverseGamma_stat.predict(test_stat_M)

In [17]:
#Оценка качества регрессии первого параметра
mape = np.mean(np.abs((test_Y[:,0] - Cat_predictions[:,0]) / test_Y[:,0])) * 100
r2 = r2_score(test_Y[:,0], Cat_predictions[:,0])
mse = mean_squared_error(test_Y[:,0], Cat_predictions[:,0])
rmse = np.sqrt(mean_squared_error(test_Y[:,0], Cat_predictions[:,0]))
print("\nStat par 1")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')

#Оценка качества регрессии второго параметра
mape = np.mean(np.abs((test_Y[:,1] - Cat_predictions[:,1]) / test_Y[:,1])) * 100
r2 = r2_score(test_Y[:,1], Cat_predictions[:,1])
mse = mean_squared_error(test_Y[:,1], Cat_predictions[:,1])
rmse = np.sqrt(mean_squared_error(test_Y[:,1], Cat_predictions[:,1]))
print("\nStat par 2")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')


Stat par 1
MSE: 0.002153446612597196
RMSE: 0.046405243373967946
R² Score: 0.9883507098902653
MAPE: 1.5657510532641115%

Stat par 2
MSE: 0.0001452777182996729
RMSE: 0.012053120687177777
R² Score: 0.9982932870674506
MAPE: 2.2559586853078257%


### Регрессия на моментах

In [18]:
train_m_X = train_X.copy()
for i in range(train_X.shape[0]):
    for j in range(1, train_X.shape[1]):
        train_m_X[i,j] += train_m_X[i,j-1]

test_m_X = test_X.copy()
for i in range(test_X.shape[0]):
    for j in range(1, test_X.shape[1]):
        test_m_X[i,j] += test_m_X[i,j-1]

In [19]:
InverseGamma_m = CatBoostRegressor(loss_function='MultiRMSE')
InverseGamma_m.fit(train_m_X, train_Y, eval_set = (test_m_X, test_Y))

0:	learn: 0.5131430	test: 0.5136565	best: 0.5136565 (0)	total: 2.64s	remaining: 43m 54s
1:	learn: 0.5073781	test: 0.5078248	best: 0.5078248 (1)	total: 4.84s	remaining: 40m 13s
2:	learn: 0.5018752	test: 0.5022471	best: 0.5022471 (2)	total: 7.17s	remaining: 39m 42s
3:	learn: 0.4966312	test: 0.4969265	best: 0.4969265 (3)	total: 9.33s	remaining: 38m 42s
4:	learn: 0.4915990	test: 0.4918538	best: 0.4918538 (4)	total: 11.5s	remaining: 38m 5s
5:	learn: 0.4868451	test: 0.4870551	best: 0.4870551 (5)	total: 13.7s	remaining: 37m 49s
6:	learn: 0.4823052	test: 0.4824854	best: 0.4824854 (6)	total: 16s	remaining: 37m 50s
7:	learn: 0.4779566	test: 0.4781269	best: 0.4781269 (7)	total: 18.3s	remaining: 37m 46s
8:	learn: 0.4737968	test: 0.4739526	best: 0.4739526 (8)	total: 20.5s	remaining: 37m 38s
9:	learn: 0.4698303	test: 0.4699535	best: 0.4699535 (9)	total: 22.7s	remaining: 37m 28s
10:	learn: 0.4660844	test: 0.4661345	best: 0.4661345 (10)	total: 24.8s	remaining: 37m 13s
11:	learn: 0.4625859	test: 0.4625

In [20]:
Cat_predictions = InverseGamma_m.predict(test_X)

In [21]:
InverseGamma_m.save_model('InverseGamma_moment_model.cbm')

In [22]:
#Оценка качества регрессии первого параметра
mape = np.mean(np.abs((test_Y[:,0] - Cat_predictions[:,0]) / test_Y[:,0])) * 100
r2 = r2_score(test_Y[:,0], Cat_predictions[:,0])
mse = mean_squared_error(test_Y[:,0], Cat_predictions[:,0])
rmse = np.sqrt(mean_squared_error(test_Y[:,0], Cat_predictions[:,0]))
print("\nMoment par 1")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')

#Оценка качества регрессии второго параметра
mape = np.mean(np.abs((test_Y[:,1] - Cat_predictions[:,1]) / test_Y[:,1])) * 100
r2 = r2_score(test_Y[:,1], Cat_predictions[:,1])
mse = mean_squared_error(test_Y[:,1], Cat_predictions[:,1])
rmse = np.sqrt(mean_squared_error(test_Y[:,1], Cat_predictions[:,1]))
print("\nMoment par 2")
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')
print(f'R² Score: {r2}')
print(f'MAPE: {mape}%')


Moment par 1
MSE: 0.47645847342150205
RMSE: 0.6902597144709388
R² Score: -1.5774509336148603
MAPE: 29.69341054220656%

Moment par 2
MSE: 0.31703440485803597
RMSE: 0.5630580830234444
R² Score: -2.724499015865452
MAPE: 414.2401955549103%
